1\. Write a function that converts numbers among the bin, dec, and hex representations (bin<->dec<->hex)

In [11]:
# input number and its base
def change_rep(n, base):
    if base == 2:
        n_bin = n
        n_dec = int(n, 2)
        n_hex = hex(n_dec)
    elif base == 10:
        n_bin = bin(n)
        n_dec = n
        n_hex = hex(n)
    elif base == 16:
        n_dec = int(n, 16)
        n_bin = bin(n_dec)
        n_hex = hex(n_dec)
    else:
        print("Aborting: invalid base.")
        return None
    print("n_2 =", n_bin, "\nn_10 =", n_dec, "\nn_16 =", n_hex)
    return(n_bin, n_dec, n_hex)

    
# example
n = "1100"
change_rep(n, 2)
print("---")
n = "fb"
change_rep(n, 16)
print("---")
n = 15
change_rep(n, 10)

n_2 = 1100 
n_10 = 12 
n_16 = 0xc
---
n_2 = 0b11111011 
n_10 = 251 
n_16 = 0xfb
---
n_2 = 0b1111 
n_10 = 15 
n_16 = 0xf


('0b1111', 15, '0xf')

2\. Write a function that converts a 32 bit word into a single precision floating point (i.e. interprets the various bits as sign, mantissa and exponent) and check its validity.

In [12]:
# the function accepts a 32-chars string
def bfloat(n):
    if len(n) != 32:
        print("Aborting: not a 32 bit string.")
        return None
    
    sign = (-1)**int(n[0], 2)
    exponent = int(n[1:9], 2)
    m = n[9:]
    f = 0
    for i in range(0, len(m)):
        f += int(m[i]) * 2**(-i-1)
    mantissa = 1 + f
    res = sign * 2**(exponent-127) * mantissa
    
    return res
     
# example  
n = "11000000101100000000000000000000"
bfloat(n)

-5.5

3\. Write a program to determine the underflow and overflow limits (within a factor of 2) for python on your computer. 

**Tip**: define two variables initialized to 1 and halve/double them for a sufficient amount of times to exceed the under/over-flow limits  

In [13]:
a = 1.0
b = 1.0
notFound = True

# determine underflow
while notFound:
    c = a / 2
    if c > 0:
        a /= 2
    else:
        notFound = False
        print("Underflow =", a)

# determine overflow
notFound = True

while notFound:
    d = b * 2
    if d != float("inf"):
        b *= 2
    else:
        notFound = False
        print("Overflow =", b)

Underflow = 5e-324
Overflow = 8.98846567431158e+307


In [14]:
# check information about float in my system
import sys
print(sys.float_info)

sys.float_info(max=1.7976931348623157e+308, max_exp=1024, max_10_exp=308, min=2.2250738585072014e-308, min_exp=-1021, min_10_exp=-307, dig=15, mant_dig=53, epsilon=2.220446049250313e-16, radix=2, rounds=1)


4\. Write a program to determine the machine precision

**Tip**: define a new variable by adding an increasingly smaller value (in the same way as the previous problem) and check when the addition starts to have no effect on the number

In [15]:
notFound = True
before, after, i = 0, 0, 0

while notFound:
    after += 10**(-i)
    if before < after:
        before = after
        i += 1
    else:
        notFound = False
        minPower = i

print("Machine precision is equal to", 10**(-minPower))

Machine precision is equal to 1e-16


5\. Write a function that takes in input three parameters $a$, $b$ and $c$ and prints out the two solutions to the quadratic equation $ax^2+bx+c=0$ using the standard formula:
$$
x=\frac{-b\pm\sqrt{b^2-4ac}}{2a}
$$

(a) use the program to compute the solution for $a=0.001$, $b=1000$ and $c=0.001$

(b) re-express the standard solution formula by multiplying the numerator and the denominator by $-b\mp\sqrt{b^2-4ac}$ and again find the solution for $a=0.001$, $b=1000$ and $c=0.001$. How does it compare with what has been previously obtained, and why?

(c) write a function that computes the roots of a quadratic equation accurately in all cases

In [16]:
# retireve sqrt function
from math import sqrt

# define the parabola function
def parabola(a, b, c, x):
    return a*x**2 + b*x + c

# parameters of the parabola
a, b, c = 0.001, 1000, 0.001

In [17]:
'''
    Solution (a).
'''
def findSol_1(a, b, c):
    val = sqrt( b**2 - 4*a*c )
    x, y = (- b + val) / (2 * a), (- b - val) / (2 * a)
    return(x, y)

# example
res = findSol_1(a, b, c)
print("Using the first formula:", res)
print("z = ax^2 + bx + c evaluated in (x, y):", (parabola(a, b, c, res[0]), parabola(a, b, c, res[1])))

Using the first formula: (-9.999894245993346e-07, -999999.999999)
z = ax^2 + bx + c evaluated in (x, y): (1.0575401665491313e-08, 7.247924804689582e-08)


In [18]:
'''
    Solution (b).
    Using the two formulas, we have to take into account 
    round-off errors when subtracting two very similar
    numbers (in this case, they are similar because b
    is way larger than a and c). That's why the result
    are slightly different.
'''
def findSol_2(a, b, c):
    val = sqrt( b**2 - 4*a*c )
    x, y =  (2 * c) / (- b - val), (2 * c) / (- b + val) 
    return(x, y)

# example
res = findSol_2(a, b, c)
print("Using the second formula:", res)
print("z = ax^2 + bx + c evaluated in (x, y):", (parabola(a, b, c, res[0]), parabola(a, b, c, res[1])))

Using the second formula: (-1.000000000001e-06, -1000010.5755125057)
z = ax^2 + bx + c evaluated in (x, y): (0.0, 10575.62534720993)


In [19]:
'''
    Solution (c).
    To avoid round-off errors I exploit the (b) 
    formula for x and the (a) formula for y.
'''
def findSol_3(a, b, c):
    val = sqrt( b**2 - 4*a*c )
    x, y = (2 * c) / (- b - val), (- b - val) / (2 * a)
    return(x, y)

# example
res = findSol_3(a, b, c)
print("Using the third formula:", res)
print("z = ax^2 + bx + c evaluated in (x, y):", (parabola(a, b, c, res[0]), parabola(a, b, c, res[1])))

Using the third formula: (-1.000000000001e-06, -999999.999999)
z = ax^2 + bx + c evaluated in (x, y): (0.0, 7.247924804689582e-08)


In [20]:
'''
    Using parameters with comparable values,
    the round-off error does not take place
    and all functions return the same results.
'''
a, b, c = 1, 3, -4

res = findSol_1(a, b, c)
print("Using the first formula:", res)
print("z = ax^2 + bx + c evaluated in (x, y):", (parabola(a, b, c, res[0]), parabola(a, b, c, res[1])))
print("---")
res = findSol_2(a, b, c)
print("Using the second formula:", res)
print("z = ax^2 + bx + c evaluated in (x, y):", (parabola(a, b, c, res[0]), parabola(a, b, c, res[1])))
print("---")
res = findSol_3(a, b, c)
print("Using the third formula:", res)
print("z = ax^2 + bx + c evaluated in (x, y):", (parabola(a, b, c, res[0]), parabola(a, b, c, res[1])))

Using the first formula: (1.0, -4.0)
z = ax^2 + bx + c evaluated in (x, y): (0.0, 0.0)
---
Using the second formula: (1.0, -4.0)
z = ax^2 + bx + c evaluated in (x, y): (0.0, 0.0)
---
Using the third formula: (1.0, -4.0)
z = ax^2 + bx + c evaluated in (x, y): (0.0, 0.0)


6\. Write a program that implements the function $f(x)=x(x−1)$

(a) Calculate the derivative of the function at the point $x = 1$ using the derivative definition:

$$
\frac{{\rm d}f}{{\rm d}x} = \lim_{\delta\to0} \frac{f(x+\delta)-f(x)}{\delta}
$$

with $\delta = 10^{−2}$. Calculate the true value of the same derivative analytically and compare it with the answer your program gives. The two will not agree perfectly. Why?

(b) Repeat the calculation for $\delta = 10^{−4}, 10^{−6}, 10^{−8}, 10^{−10}, 10^{−12}$ and $10^{−14}$. How does the accuracy scales with $\delta$?

In [21]:
'''
    Solution (a).
    The derivative definition holds as delta 
    approaches to zero. In this case, delta is quite 
    large and thus the formula overestimates the value.
'''
def f(x):
    return x**2 - x

def f_p(x):
    return 2*x - 1

def df(x, d):
    return (f(x+d) - f(x)) / d

print("True value =", f_p(1))
print("Using the definition =", df(1, 10**(-2)))

True value = 1
Using the definition = 1.0099999999999998


In [22]:
'''
    Solution (b).
    Although we might expect that as delta goes to 
    zero the definition returns a number closer to the
    true value, the function of the derivative definition
    is poorly conditioned, hence the result does not
    vary gradually as the input varies.
'''
dl = [10**(-c) for c in [2, 4, 6, 8, 10, 12, 14]]

for val in dl:
    print("Using d =", val, "the derivative is", df(1, val))

Using d = 0.01 the derivative is 1.0099999999999998
Using d = 0.0001 the derivative is 1.0000999999992821
Using d = 1e-06 the derivative is 1.000001000006634
Using d = 1e-08 the derivative is 0.999999993922529
Using d = 1e-10 the derivative is 1.000000082740371
Using d = 1e-12 the derivative is 1.000088900582341
Using d = 1e-14 the derivative is 0.9992007221626409


7\. Consider the integral of the semicircle of radius 1:
$$
I=\int_{-1}^{1} \sqrt(1-x^2) {\rm d}x
$$
which is known to be $I=\frac{\pi}{2}=1.57079632679...$.

Alternatively we can use the Riemann definition of the integral:
$$
I=\lim_{N\to\infty} \sum_{k=1}^{N} h y_k 
$$

with $h=2/N$ the width of each of the $N$ slices the domain is divided into and where
$y_k$ is the value of the function at the $k-$th slice.

(a) Write a program to compute the integral with $N=100$. How does the result compare to the true value?

(b) How much can $N$ be increased if the computation needs to be run in less than a second? What is the gain in running it for 1 minute? 


In [23]:
# retrieve pi
from math import pi

# define function
def f(x):
    return sqrt(1 - x**2)

# retrieve time function
import time

In [24]:
'''
    Solution (a).
'''
def int_f(N):
    # variables: grid spacing, starting point
    h = 2 / N
    x0 = -1
    
    # integral 
    integral = 0
    for k in range(0, N+1):
        x = x0 + k*h
        integral += f(x)*h
        
    return integral
       
# example
N = 100
res = int_f(N)
print("Using the Riemann definition with", N, "slices:", res)
var = 100 * ((pi/2) - res) / (pi/2)
print("The result varies by " + str(var) + "% with respect to the true value.")

Using the Riemann definition with 100 slices: 1.5691342555492505
The result varies by 0.10581074180619238% with respect to the true value.


In [27]:
'''
    Solution (b).
    I use the IPython built-in function %timeit
    to predict the execution time of a function.
    The execution time varies a bit from time to
    time.
'''
print("Predicted execution time respectively for 100, 1000, 1000000 slices:")
%timeit int_f(100)
%timeit int_f(1000)
%timeit int_f(1000000)

Predicted execution time respectively for 100, 1000, 1000000 slices:
93.6 µs ± 3.93 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
915 µs ± 3.24 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
924 ms ± 15.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [28]:
# start clock
start = time.time()

# compute integral
N = 1000000
res = int_f(N)

# check time
print("Using N =", N, "the execution time is approximately of", time.time() - start, "seconds.")

# results
print("Using the Riemann definition with", N, "slices:", res)
var = 100 * ((pi/2) - res) / (pi/2)
print("The result varies by " + str(var) + "% with respect to the true value.")

Using N = 1000000 the execution time is approximately of 1.0170669555664062 seconds.
Using the Riemann definition with 1000000 slices: 1.5707963251317274
The result varies by 1.058806370743233e-07% with respect to the true value.


In [29]:
'''
    Assuming that N(time) is approximately linear and
    using the previous results, one can predict that
    if N = 60e+06 the execution time is approximately 
    equal to 60 seconds. 
    In this case, the variation between result and true value 
    is o(1e-10 %), three orders better than the variation between
    the result obtained with 1 second and the true value, which was o(1e-07 %).
'''
# start clock
start = time.time()

# compute integral
N = 60000000
res = int_f(N)

# check time
print("Time =", time.time() - start)

# results
print("Using the Riemann definition with", N, "slices:", res)
var = 100 * ((pi/2) - res) / (pi/2)
print("The result varies by " + str(var) + "% with respect to the true value.")

Time = 55.3774995803833
Using the Riemann definition with 60000000 slices: 1.5707963267911924
The result varies by 2.358133919829973e-10% with respect to the true value.
